In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"]="false"

import pathlib
import glob
from copy import deepcopy

import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import jax
import jax.numpy as jnp
import equinox as eqx

In [ ]:
from mc2.utils.data_inspection import (
    get_available_material_names,
    get_file_overview,
    load_and_process_single_from_full_file_overview
)
from mc2.data_management import FrequencySet, MaterialSet, DataSet

## load data from pickle:

In [ ]:
LOAD_ALL_MATERIALS = False
SAVE_SINGLE_PICKLE = False

if LOAD_ALL_MATERIALS:
    dataset = DataSet.load_from_file(pathlib.Path("../../data/processed") / "ten_mat_data.pickle")

    ## deleting N49 from dataset for now, since the data is incomplete
    # 50 kHz and 80 kHz are missing
    # 320 kHz has no data at 25 degrees

    available_materials = deepcopy(dataset.material_names)
    print(available_materials)
    print(len(available_materials))

    available_materials.remove("N49")
    print(available_materials)
    print(len(available_materials))
  
    if SAVE_SINGLE_PICKLE:
        dataset_3C90 = dataset.filter_materials(['3C90'])
        dataset_3C90.save_to_file(pathlib.Path("../../data/processed") / "3C90.pickle")  

In [ ]:
LOAD_3C90 = True
dataset = DataSet.load_from_file(pathlib.Path("../../data/processed") / "3C90.pickle")


In [ ]:
data_3C90 = dataset.filter_materials(material_names="3C90")
data_3C90_25_C = data_3C90.filter_temperatures([25])

for frequency_set in data_3C90_25_C.material_sets[0].frequency_sets:
    print(frequency_set.frequency)
    # for i in range(0, frequency_set.H.shape[0], 100):
    #     h, b, f = frequency_set.H[i], frequency_set.B[i], frequency_set.frequency
    #     plt.plot(h/max(h))
    #     # plt.plot(b/max(b))
    #     plt.ylim(-1/256, 1/256)
    #     # plt.hist(h, 2**8)
    #     # plt.xlim(-3, 3)
    #     plt.show()

In [ ]:
# Filter from full dataset for a single material and temperature
data_3C90 = dataset.filter_materials(material_names="3C90")
data_3C90_25_C = data_3C90.filter_temperatures([25])


In [ ]:
# Find out, 
# ...if the DC voltage is constant; slope: b_min, b_max and D*Ts
# ...Neukurve extrahieren


# Plot the dutycycle (over time)
data_3C90_25_C
for frequency_set in data_3C90_25_C.material_sets[0].frequency_sets:
    # print(frequency_set.H.shape[0])
    for i in range(0, frequency_set.H.shape[0], 2000):

        # A single "PFC" sequence:
        h, b, f = frequency_set.H[i], frequency_set.B[i], frequency_set.frequency
        dbdt = np.gradient(b)
        sign = np.sign(dbdt)

        # plt.plot(dbdt)
        plt.plot(sign[2000:4000])

        # plt.plot(h, b)
        plt.show()

In [ ]:
# Plots of where the switching events occur
# Counting of the withing events


PLOT_HYSTERESIS = True
PLOT_SWITCHING_EVENTS = True
PLOT_RIPPLE_HIST = False

for frequency_set in data_3C90_25_C.material_sets[0].frequency_sets:
    print(frequency_set)
    for i in range(0, frequency_set.H.shape[0], 100):
        h, b, f = frequency_set.H[i], frequency_set.B[i], frequency_set.frequency
        if PLOT_HYSTERESIS:
            plt.plot(h, b)

        # postive, if db/dt is positive and 
        bool_ascending = np.roll(b, shift=1) - b
        # Get the sign of each element
        signs = np.sign(bool_ascending)
        # Compute differences between adjacent signs
        sign_changes = np.diff(signs, append=signs[0])  # append the first value to keep the same dimension


        # Count non-zero sign changes
        # always 198 ...
        count_sign_changes = np.count_nonzero(sign_changes)
        print("\nSign changes of db/dt:")
        print(f"No. of 1: {count_sign_changes} (sign changes)")
        print(f"No. of 0: {len(sign_changes)-count_sign_changes}\n")


        # Carrier signal with "slow" frequency
        print("Carrier signal:")
        T_carrier = 1 / f * count_sign_changes / 2
        print(f"{T_carrier = }")
        f_carrier = 1 / T_carrier
        print(f"{f_carrier = }")
        print(f"hmax = {float(max(h))}")
        print(f"bmax = {float(max(b))}\n")


        # Find the amplitude:
        mask = (sign_changes != 0).astype(int)  # this is 1, if there is the beginning or the end of a "fast" cycle

        b_HF_amplitudes = np.where(mask, b, 0)

        b_HF_amplitudes_filtered = b_HF_amplitudes[b_HF_amplitudes != 0]
        b_ripple = abs(np.roll(b_HF_amplitudes_filtered, shift=-1) - b_HF_amplitudes_filtered)


        if PLOT_RIPPLE_HIST:
            # Overimposed "fast" ripple
            print("Overimposed ripple:")
            plt.hist(b_ripple)

        if PLOT_SWITCHING_EVENTS:
            plt.plot(h, mask/10, label="switching events") # db/dt sign changes 

        plt.legend()
        plt.show()